In [12]:
!which python

/Users/sid/Desktop/Personel/SJSU/Semester III/CMPE 258- DL/Project/virtual_env/bin/python


In [1]:
# Install MediaPipe
!pip install mediapipe

# Install joblib (usually pre-installed, but just in case)
!pip install joblib

# Install tqdm for progress bars
!pip install tqdm

# If you need a specific version of OpenCV
!pip install opencv-python


  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/67/95/30e3aa1d00609c01435512aeddfc2eda694a24f8ff45c287a61f45fb6f3c/mediapipe-0.10.18-cp312-cp312-macosx_11_0_universal2.whl.metadata
  Obtaining dependency information for absl-py from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Obtaining dependency information for attrs>=19.1.0 from https://files.pythonhosted.org/packages/6a/21/5b6702a7f963e95456c0de2d495f67bf5fd62840ac655dc451586d23d39a/attrs-24.2.0-py3-none-any.whl.metadata
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Obtaining dependency information for flatbuffers>=2.0 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2dbd0ad4182ef2d53488bb02e58694cd79d61369e85900/flatbuffers-24.3.25-py2.py3-none-any.whl.metadata
  Using cached flatbuffers

In [9]:
INPUT_VIDEO_PATH = "input_videos/pushup_010.mp4"
OUTPUT_VIDEO_PATH = "output_videos/pushup_010.mp4"

# Model & Testing Paths
MODEL_PATH = 'trained_model_files/final_trained_model.pth'
SCALER_PATH = 'trained_model_files/scaler.pkl'

In [10]:
import os

root_path = os.path.dirname(os.path.abspath(''))
print(root_path)

INPUT_VIDEO_PATH = os.path.join(root_path, INPUT_VIDEO_PATH)
OUTPUT_VIDEO_PATH = os.path.join(root_path, OUTPUT_VIDEO_PATH)

# Model & Testing Paths
MODEL_PATH = os.path.join(root_path, MODEL_PATH)
SCALER_PATH = os.path.join(root_path, SCALER_PATH)

print(INPUT_VIDEO_PATH)
print(OUTPUT_VIDEO_PATH)
print(MODEL_PATH)
print(SCALER_PATH)



/Users/sid/Desktop/Personel/SJSU/Semester III/CMPE 258- DL/Project
/Users/sid/Desktop/Personel/SJSU/Semester III/CMPE 258- DL/Project/input_videos/pushup_010.mp4
/Users/sid/Desktop/Personel/SJSU/Semester III/CMPE 258- DL/Project/output_videos/pushup_010.mp4
/Users/sid/Desktop/Personel/SJSU/Semester III/CMPE 258- DL/Project/trained_model_files/final_trained_model.pth
/Users/sid/Desktop/Personel/SJSU/Semester III/CMPE 258- DL/Project/trained_model_files/scaler.pkl


In [11]:
# Import necessary libraries
import os
import cv2
import mediapipe as mp
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import joblib
from tqdm import tqdm

# Define the model architecture (must match the training architecture)
class Conv3DBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3,
                 stride=1, padding=1):
        super().__init__()
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm3d(out_channels)
        self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = Conv3DBlock(in_channels, out_channels)
        self.conv2 = Conv3DBlock(out_channels, out_channels)
        self.downsample = None
        if in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv3d(in_channels, out_channels, kernel_size=1, stride=1),
                nn.BatchNorm3d(out_channels)
            )
        self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out = out + identity
        return self.relu(out)

class ResNet3D(nn.Module):
    def __init__(self, block, layers, num_classes=2, input_channels=8):
        super().__init__()
        self.in_channels = 64
        self.conv1 = Conv3DBlock(input_channels, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        layers = []
        layers.append(block(self.in_channels, out_channels))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = x.unsqueeze(2).unsqueeze(-1)  # Shape: [batch_size, channels, 1, sequence_length, 1]
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

def resnet3d18(num_classes=2, input_channels=8):
    return ResNet3D(ResidualBlock, [2, 2, 2, 2], num_classes, input_channels)

# Load the trained model
model = resnet3d18(num_classes=2, input_channels=8)
model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cpu')))
model.eval()

# Load the scaler
scaler = joblib.load(SCALER_PATH)

# Initialize MediaPipe Pose for pose estimation
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

def calculate_angle(a, b, c):
    """Calculate the angle between three points."""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - \
              np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return float(angle)

def extract_features(frame):
    """Extract features from a single frame."""
    return [
        frame['left_arm_angle'],
        frame['right_arm_angle'],
        frame['landmarks'][11][1],  # LEFT_SHOULDER y-coordinate
        frame['landmarks'][12][1],  # RIGHT_SHOULDER y-coordinate
        frame['landmarks'][13][1],  # LEFT_ELBOW y-coordinate
        frame['landmarks'][14][1],  # RIGHT_ELBOW y-coordinate
        frame['landmarks'][15][1],  # LEFT_WRIST y-coordinate
        frame['landmarks'][16][1]   # RIGHT_WRIST y-coordinate
    ]

def preprocess_video(video_path, sequence_length=100):
    """Preprocess the video and extract features."""
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    preprocessed_data = []
    frames = []  # Store frames for visualization

    for frame_idx in tqdm(range(frame_count), desc="Processing Video"):
        ret, frame = cap.read()
        if not ret:
            break

        frames.append(frame.copy())  # Save the frame for later visualization

        # Convert the image to RGB and process it with MediaPipe
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Extract relevant landmarks
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            # Calculate angles
            left_arm_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_arm_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

            preprocessed_data.append({
                'frame': frame_idx,
                'left_arm_angle': left_arm_angle,
                'right_arm_angle': right_arm_angle,
                'landmarks': [(landmark.x, landmark.y, landmark.z) for landmark in landmarks]
            })
        else:
            # If no landmarks detected, append zeros
            preprocessed_data.append({
                'frame': frame_idx,
                'left_arm_angle': 0.0,
                'right_arm_angle': 0.0,
                'landmarks': [(0.0, 0.0, 0.0)] * 33  # Assuming 33 landmarks
            })

    cap.release()
    return preprocessed_data, frames

def visualize_results(frames, predicted_class, confidence):
    """Overlay the prediction on video frames and save the result."""
    # Define the label and color based on the prediction
    if predicted_class == 1:
        label = f'Correct Form ({confidence * 100:.2f}%)'
        color = (0, 255, 0)  # Green for correct
    else:
        label = f'Incorrect Form ({confidence * 100:.2f}%)'
        color = (0, 0, 255)  # Red for incorrect

    # Initialize video writer
    height, width, _ = frames[0].shape
    output_path = OUTPUT_VIDEO_PATH
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'),
                          20, (width, height))

    for frame in frames:
        # Overlay text on the frame
        cv2.putText(frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                    1.5, color, 3)
        out.write(frame)

    out.release()
    print(f"Visualization saved as {output_path}")

def main():
    # Path to your new video file
    video_path = INPUT_VIDEO_PATH  # Update this path if necessary

    # Preprocess the video and extract features
    preprocessed_data, frames = preprocess_video(video_path)

    # Extract features from preprocessed data
    features = np.zeros((len(preprocessed_data), 8))
    for i, frame in enumerate(preprocessed_data):
        features[i] = extract_features(frame)

    # Normalize features using the saved scaler
    features_normalized = scaler.transform(features)

    # Standardize sequence length
    sequence_length = 100  # Same as during training
    if len(features_normalized) < sequence_length:
        pad_length = sequence_length - len(features_normalized)
        features_normalized = np.pad(features_normalized, ((0, pad_length), (0, 0)), mode='constant')
    elif len(features_normalized) > sequence_length:
        features_normalized = features_normalized[:sequence_length]

    # Prepare input tensor
    input_tensor = torch.FloatTensor(features_normalized).permute(1, 0).unsqueeze(0)

    # Make prediction
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = torch.softmax(output, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0, predicted_class].item()

    # Output result
    if predicted_class == 1:
        print(f"The push-up form is **correct** with confidence {confidence * 100:.2f}%")
    else:
        print(f"The push-up form is **incorrect** with confidence {confidence * 100:.2f}%")

    # Visualize the results
    visualize_results(frames, predicted_class, confidence)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'torch'